In [16]:
import pandas as pd 
import numpy as np 
import sklearn.preprocessing
import imblearn

In [17]:
dfTrain = pd.read_csv("train.csv")
dfTest = pd.read_csv("test.csv")
testLabels = pd.read_csv("submission.csv")

In [23]:
dfTrain
X_train, y_train = dfTrain.loc[:, dfTrain.columns[:-1]], dfTrain["subscribed"]
X_test = dfTest.loc[:, dfTest.columns]

In [24]:
X_train.drop("id", axis=1, inplace=True)
X_test.drop("id", axis=1, inplace=True)

In [9]:
def sin_transformer(period):
	return sklearn.preprocessing.FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
	return sklearn.preprocessing.FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

def encodingPipeline(X_train): # Should just transform for X_test, for now it also fits it
    X_category_encoded = pd.get_dummies(X_train, columns=['marital', 'loan', 'housing', 'poutcome'], drop_first=True)
    jobEncoder = sklearn.preprocessing.OrdinalEncoder(categories=[["unknown", "unemployed", "student", "housemaid", "services", "admin.", "technician", "self-employed", "retired", "entrepreneur", "management", "blue-collar"]])
    X_category_encoded["job"] = jobEncoder.fit_transform(np.array(X_category_encoded["job"]).reshape(-1, 1))
    eduEncoder = sklearn.preprocessing.OrdinalEncoder(categories=[["unknown", "illiterate", "basic.4y", "basic.6y", "basic.9y", "high.school", "professional.course", "university.degree"]])
    X_category_encoded["education"] = eduEncoder.fit_transform(np.array(X_category_encoded["education"]).reshape(-1, 1))
    binaryEncoder = sklearn.preprocessing.OrdinalEncoder()
    X_category_encoded["contact"] = binaryEncoder.fit_transform(np.array(X_category_encoded["contact"]).reshape(-1, 1))
    monthEncoder = sklearn.preprocessing.OrdinalEncoder(categories=[["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]])
    X_category_encoded["month"] = monthEncoder.fit_transform(np.array(X_category_encoded["month"]).reshape(-1, 1))
    dayEncoder = sklearn.preprocessing.OrdinalEncoder(categories=[["mon", "tue", "wed", "thu", "fri"]])
    X_category_encoded["day_of_week"] = dayEncoder.fit_transform(np.array(X_category_encoded["day_of_week"]).reshape(-1, 1))
    X_category_encoded["default"] = binaryEncoder.fit_transform(np.array(X_category_encoded["default"]).reshape(-1, 1))

    X_category_encoded["month_sin"] = sin_transformer(12).fit_transform(X_category_encoded)["month"]
    X_category_encoded["month_cos"] = cos_transformer(12).fit_transform(X_category_encoded)["month"]

    X_category_encoded["day_sin"] = sin_transformer(5).fit_transform(X_category_encoded)["day_of_week"]
    X_category_encoded["day_cos"] = cos_transformer(5).fit_transform(X_category_encoded)["day_of_week"]
    X_category_encoded.drop(labels=["month", "day_of_week"], inplace=True, axis=1)
    return X_category_encoded
    

    

In [25]:
categoryCols = X_train.select_dtypes(exclude=np.number).columns
binaryEncoder = sklearn.preprocessing.OrdinalEncoder()
y_train = binaryEncoder.fit_transform(np.array(y_train).reshape(-1, 1))

In [30]:
y_train.ravel()

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:

X_train_numeric = X_train.select_dtypes(include=np.number)
X_train_encoded = encodingPipeline(X_train[categoryCols])
X_train = pd.concat([X_train_numeric, X_train_encoded], axis=1)

In [26]:
smotenc = imblearn.over_sampling.SMOTENC(categorical_features=categoryCols, n_jobs=-1)

In [28]:
X_train_res, y_train_res = smotenc.fit_resample(X_train, y_train.ravel())

d:\Anaconda\Lib\site-packages\imblearn\over_sampling\_smote\base.py:645: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()